In [12]:
import pickle

In [13]:
training_data_7k = []
with open('training_data_7k.pkl' ,'rb')as f :
    training_data_7k = pickle.load(f)

In [15]:
# training_data_7k[0]

In [16]:
import multiprocessing
from gensim.models import Word2Vec
import time
import numpy as np
from numpy.linalg import norm

In [17]:
cores = multiprocessing.cpu_count()
print('Number of cores in this device {}'.format(cores))

Number of cores in this device 8


In [18]:
docs_num = len(training_data_7k)
tokens_num = sum([len(x) for x in training_data_7k])
print('Number of all docs {}'.format(docs_num))
print('Number of all Tokens {}'.format(tokens_num))

Number of all docs 7562
Number of all Tokens 1467075


In [19]:
%%time
w2v_model = Word2Vec(min_count=1 , window=15 , vector_size=300 , alpha=0.03 , workers=cores-1)
w2v_model.build_vocab(training_data_7k)
w2v_model_vocab_size = len(w2v_model.wv)
print('Vocab size : ' , w2v_model_vocab_size)

Vocab size :  40507
Wall time: 1.72 s


In [20]:
start = time.time()
w2v_model.train(training_data_7k , total_examples=w2v_model.corpus_count , epochs= 30)
end = time.time()
print("{} s". format(end-start))

50.472068071365356 s


In [ ]:
docs_tfidf_11k = []
with open('tfidf_50k.pkl' ,'rb')as f :
    docs_tfidf_11k = pickle.load(f)

In [21]:
w2v_model.save("w2v_7k_300d.model")

In [22]:
docs_tfidf_7k = []
with open('tfidf_7k.pkl' ,'rb')as f :
    docs_tfidf_7k = pickle.load(f)

In [25]:
# docs_tfidf_7k[0]

In [26]:
def make_docs_embeding(docs_tfidf):
    docs_embedding = []
    for doc in docs_tfidf:
        doc_vector = np.zeros(300)
        weight_sum = 0
        if(len(doc.items())!=0):
            
       
            for token , weight in doc.items() :


                        doc_vector += w2v_model.wv[token] * weight
                        weight_sum += weight

            docs_embedding.append(doc_vector/weight_sum)
        
        
    
 
        
    return docs_embedding

In [31]:
def similarity(doc1 , doc2) :
    similarity_score = np.dot(doc1 , doc2) / ((norm(doc1)) *(norm(doc2)) )
    return (similarity_score +1)/2

In [32]:
s = similarity(docs_embedding_7k[11] , docs_embedding_7k[10])
s

0.849415371609672

# KNN implementation

In [36]:
import xlrd
data_set = '50k_combined.xlsm'
data_reader = xlrd.open_workbook(data_set)
content = data_reader.sheet_by_index(0)

str

In [43]:
def KNN (k , train_embedding , test_embedding) :
    #sport , economic , political , culture , health
    classes = [[],[],[],[],[]]
    
    for index ,test_doc in enumerate(test_embedding) :
        distances = {}
        for train_doc in train_embedding :
            dist = numpy.linalg.norm(test_doc-train_doc)
            distances[index] = dist
        
        distances = dict(sorted(distances.items(), key=lambda item: item[1], reverse=True))
        k_first = dict(list(distances.items())[0: k])
        
        categories_repeat = {}
        
        for doc in k_first :
            cat = data_reader.sheet_by_index(0).cell(int(d), 2).value
            if(cat not in categories_repeat) :
                categories_repeat[cat] = 1
            else :
                categories_repeat[cat] += 1
        
        categories_repeat = dict(sorted(categories_repeat.items(), key=lambda item: item[1], reverse=True))
        best_category= list(categories_repeat)[0]
        if(best_category == 'sport') :
            classes[0].append((test_doc , index))
        elif(best_category == 'economy') :
            classes[1].append((test_doc , index))
        elif(best_category == 'politics') :
            classes[2].append((test_doc , index))
        elif(best_category == 'culture') :
            classes[3].append((test_doc , index))
        elif(best_category == 'health') :  
            classes[4].append((test_doc , index))


                
    classificitaion_7k={'classes' : classes}
    with open('classification_7k.pkl' , 'wb') as f :
        pickle.dump(classificitaion_7k,f)
    
        

In [38]:
with open('docs_embedding_50k.pkl' ,'rb')as f :
    docs_embedding_50k = pickle.load(f)

In [41]:
import hazm
hazm_normalaizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()

In [42]:
from collections import Counter
import math
def query_embeding(query ,docs_num) :
       
        normal_query = hazm_normalaizer.normalize(query)
        tokens = hazm.word_tokenize(normal_query)
        tokens_stems = []
        for t in tokens :
            stem_token = lemmatizer.lemmatize(t)
            if("#" in stem_token):
                mazi , mozare = stem_token.split("#")
                tokens_stems.append(mazi)
                tokens_stems.append(mozare)
            else :
                tokens_stems.append(stem_token)
            
        token_tfidf = {}

        
        for term, count in Counter(tokens_stems).items():
            tf = (1 + math.log10(count))
            idf = math.log10(docs_num/1)
            tfidf = tf * idf
            token_tfidf [term] = tfidf
        
        query_vector = np.zeros(300)
        weight_sum = 0
        for token , weight in token_tfidf.items() :
            try:
            
                query_vector += w2v_model.wv[token] * weight
                
            except KeyError:
                continue
            weight_sum += weight
        if weight_sum == 0 :
            return query_vector
 
     
        return query_vector/weight_sum

In [ ]:
def answer_query(query , cat):
     with open('classification_7k.pkl' ,'rb')as f :
        classification_7k = pickle.load(f)
        f.close()
    if(cat == 'sport') :
        index_search = 0
    elif(cat == 'economy') :
        index_search = 1
    elif(cat == 'politics') :
        index_search = 2
    
    elif(cat == 'culture') : 
        index_search = 3
    elif(cat == 'health') : 
        index_search = 4
    
    